In [1]:
import math
import time
import wandb
import spam
import pprint

In [2]:
WANDB_PROJECT = 'matrix-factorization'
wandb.login()

wandb: Currently logged in as: lubomirkurcak (use `wandb login --relogin` to force relogin)


True

In [3]:
# vytvorenie sweep configu (pouzivame nahodne zvolenie parametrov)
sweep_config = {
    'method': 'random'
}

In [4]:
#metric = {
#    'name': 'loss',
#    'goal': 'minimize'
#}

metric = {
    'name': 'squared_error',
    'goal': 'minimize'
}

sweep_config['metric'] = metric

In [5]:
# diskretne parametre
parameters_dict = {
    'fp_subnormal': {
        'values': ['CPFLOAT_SUBN_RND',
                   'CPFLOAT_SUBN_USE']
    },
    
    'fp_round': {
        'values': ['CPFLOAT_RND_NA',
                   'CPFLOAT_RND_NZ',
                   'CPFLOAT_RND_NE',
                   'CPFLOAT_RND_TP',
                   'CPFLOAT_RND_TN',
                   'CPFLOAT_RND_TZ',
                   'CPFLOAT_RND_SP',
                   'CPFLOAT_RND_SE',
                   'CPFLOAT_RND_OD',
                   'CPFLOAT_NO_RND']
    },
    
    'fp_flip': {
        'values': ['CPFLOAT_NO_SOFTERR',
                   'CPFLOAT_SOFTERR']
    },
    
    'fp_explim': {
        'values': ['CPFLOAT_EXPRANGE_STOR',
                   'CPFLOAT_EXPRANGE_TARG']
    },
}

sweep_config['parameters'] = parameters_dict

In [6]:
# konstantne parametre
parameters_dict.update({
    'features': {
        'value': 15
    },
    'epochs': {
        'value': 50000
    },
    'non_negative': {
        'value': 1
    },
    'learning_rate': {
        'value': 0.001
    },
    'weight_decay': {
        'value': 0.0001
    },
    'dropout': {
        'value': 0
    },
    'momentum_coefficient': {
        'value': 0
    },
    'loss': {
        'value': 'mean_squared_error'
    }
})

In [7]:
'''
precision
   * @brief Bits of precision of target format.
   *
   * @details The maximum values allowed are 24 and 53 if the storage format is
   * `float` or `double`, respectively.
'''

'''
maximum exponent
   * @brief Maximum exponent of target format.
   *
   * @details The maximum values allowed are 127 and 1023 if the storage format
   * is `float` or `double`, respectively. Larger values are reduced to the
   * maximum allowed value without warning. This field is ignored unless
   * `explim` is set to `CPFLOAT_EXPRANGE_TARG`.   
'''

# spojite parametre
parameters_dict.update({
    'fp_p': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.5
    },
    
    'fp_precision': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 24,
    },
    
    'fp_emax': {
        'distribution': 'q_uniform',
        'q': 1,
        'min': 4,
        'max': 127,
    },
})


'''
'fp_precision': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(24),
},

'fp_emax': {
    'distribution': 'q_log_uniform',
    'q': 1,
    'min': math.log(4),
    'max': math.log(127),
},
'''

"\n'fp_precision': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(24),\n},\n\n'fp_emax': {\n    'distribution': 'q_log_uniform',\n    'q': 1,\n    'min': math.log(4),\n    'max': math.log(127),\n},\n"

In [8]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'squared_error'},
 'parameters': {'dropout': {'value': 0},
                'epochs': {'value': 50000},
                'features': {'value': 15},
                'fp_emax': {'distribution': 'q_uniform',
                            'max': 127,
                            'min': 4,
                            'q': 1},
                'fp_explim': {'values': ['CPFLOAT_EXPRANGE_STOR',
                                         'CPFLOAT_EXPRANGE_TARG']},
                'fp_flip': {'values': ['CPFLOAT_NO_SOFTERR',
                                       'CPFLOAT_SOFTERR']},
                'fp_p': {'distribution': 'uniform', 'max': 0.5, 'min': 0},
                'fp_precision': {'distribution': 'q_uniform',
                                 'max': 24,
                                 'min': 4,
                                 'q': 1},
                'fp_round': {'values': ['CPFLOAT_RND_NA',
                                        'C

In [9]:
spam.system('echo %TIME%')

0

In [10]:
# trenovacia funkcia, pouzita pri volani trenovacich agentov
def train():
    
    # inicialuzuj wandb a vyber config s nastavenymi hodnotami podla sweep configu
    run = wandb.init()
    config = run.config
    
    squared_error = spam.matrix_factorization(features=config["features"],
        epochs=config["epochs"],
        non_negative=config["non_negative"],
        learning_rate=config["learning_rate"],
        weight_decay=config["weight_decay"],
        dropout=config["dropout"],
        momentum_coefficient=config["momentum_coefficient"],
        fp_precision=config["fp_precision"],
        fp_emax=config["fp_emax"],
        fp_subnormal=config["fp_subnormal"],
        fp_round=config["fp_round"],
        fp_flip=config["fp_flip"],
        fp_p=config["fp_p"],
        fp_explim=config["fp_explim"]
    )
    
    wandb.log({'squared_error': squared_error}, commit=False, step=0)
        
    run.finish()
    

In [ ]:
#sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)
sweep_id = "jneupcgr"

wandb.agent(sweep_id=sweep_id, project=WANDB_PROJECT, function=train, count=400)

Create sweep with ID: jneupcgr
Sweep URL: https://wandb.ai/lubomirkurcak/matrix-factorization/sweeps/jneupcgr


wandb: Agent Starting Run: gb0lrb34 with config:
wandb: 	dropout: 0
wandb: 	epochs: 50000
wandb: 	features: 15
wandb: 	fp_emax: 56
wandb: 	fp_explim: CPFLOAT_EXPRANGE_TARG
wandb: 	fp_flip: CPFLOAT_SOFTERR
wandb: 	fp_p: 0.31903571648305157
wandb: 	fp_precision: 18
wandb: 	fp_round: CPFLOAT_RND_SP
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	learning_rate: 0.001
wandb: 	loss: mean_squared_error
wandb: 	momentum_coefficient: 0
wandb: 	non_negative: 1
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0


squared_error,184.53587
_runtime,28
_timestamp,1646180540
_step,0


squared_error,▁
_runtime,▁
_timestamp,▁
_step,▁


...

wandb: Agent Starting Run: njrv95n0 with config:
wandb: 	dropout: 0
wandb: 	epochs: 50000
wandb: 	features: 15
wandb: 	fp_emax: 115
wandb: 	fp_explim: CPFLOAT_EXPRANGE_TARG
wandb: 	fp_flip: CPFLOAT_NO_SOFTERR
wandb: 	fp_p: 0.3810444244118454
wandb: 	fp_precision: 21
wandb: 	fp_round: CPFLOAT_RND_TN
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	learning_rate: 0.001
wandb: 	loss: mean_squared_error
wandb: 	momentum_coefficient: 0
wandb: 	non_negative: 1
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


cpfloat_validate_optstruct returned 0


squared_error,174.92148
_runtime,16
_timestamp,1646180700
_step,0


squared_error,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 0out7kkh with config:
wandb: 	dropout: 0
wandb: 	epochs: 50000
wandb: 	features: 15
wandb: 	fp_emax: 110
wandb: 	fp_explim: CPFLOAT_EXPRANGE_STOR
wandb: 	fp_flip: CPFLOAT_NO_SOFTERR
wandb: 	fp_p: 0.2865503740946233
wandb: 	fp_precision: 5
wandb: 	fp_round: CPFLOAT_RND_TP
wandb: 	fp_subnormal: CPFLOAT_SUBN_USE
wandb: 	learning_rate: 0.001
wandb: 	loss: mean_squared_error
wandb: 	momentum_coefficient: 0
wandb: 	non_negative: 1
wandb: 	weight_decay: 0.0001
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
